In [1]:
import torch
import random

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, attention_mask = tokenizer.encode('how are you', max_length=4)

input_ids, attention_mask, tokenizer.decode(input_ids)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(tensor([128000,   5269,    527, 128001]),
 tensor([1, 1, 1, 1]),
 '<|begin_of_text|>how are<|end_of_text|>')

In [2]:
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset('json', data_files='dataset/train.json', split='train')

#4,2,4切分,取第0部分
dataset = dataset.select(range(30000))


def f(data):
    #随机生成两种回答
    if random.random() > 0.5:
        data['chosen'] = data['chosen'].swapcase()
    data = data['prompt'] + data['chosen']

    input_ids, attention_mask = tokenizer.encode(data)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader)).keys()

(7500, dict_keys(['input_ids', 'attention_mask', 'labels']))

In [3]:
%run 1.model.ipynb
from accelerate import Accelerator

model_actor = LlamaForCausalLM()

model_actor.train()

optimizer = torch.optim.Adam(model_actor.parameters(), lr=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                       T_max=100,
                                                       eta_min=1e-5)

accelerator = Accelerator(mixed_precision='fp16')

loader, model_actor, optimizer, scheduler = accelerator.prepare(
    loader, model_actor, optimizer, scheduler)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
for epoch in range(5):
    for i, data in enumerate(loader):
        loss, logits = model_actor(**data)
        accelerator.backward(loss)
        accelerator.clip_grad_norm_(model_actor.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

        if (i + 1) % 100 == 0:
            lr = optimizer.param_groups[-1]['lr']
            print(epoch, i, len(loader), loss.item(), lr)
            logits = logits[0].argmax(1)
            print(tokenizer.decode(logits))

torch.save(model_actor.to('cpu'), 'model/actor')

0 99 7500 1.863224983215332 1.002220478354208e-05
Human: context= CREATE TABLE table_name_ VARCHAR (name VARCHAR, date VARCHAR, question= What is the game of the? date the of the1 = Assistant? Assistant2 = =:SELECT COUNT =) from table_name_1 =  = "1" Assistant = Assistant2 =<|end_of_text|>
0 199 7500 2.670088052749634 9.997779521646018e-05
Human: context= CREATE TABLE table_name_2 (year VARCHAR VARCHAR VARCHAR, year VARCHAR VARCHAR VARCHAR,  VARCHAR)  VARCHAR) question= What is there season is?? has?6? the season? 6? the season?  when??? than 1, Assistant, Assistant:select count""ITION"" from TABLE_NAME_2 WHERE SCORE"""" "1 where DATE""" "1" DATE"""""" 1, Assistant"<|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special_token_0|><|reserved_special

0 1999 7500 0.9893758893013 9.997779521648096e-05
Human: context= CREATE TABLE table_name134418__1 (c VARCHAR VARCHAR, incumbent VARCHAR VARCHAR) question= What is the episode number for the episode score a8:1??? Assistant:SELECT COUNT FROM FROM table_134418__3 WHERE points oval = "2-1"<|end_of_text|>
0 2099 7500 1.9308842420578003 1.002220478354209e-05
Human: context= CREATE TABLE table_name_54 (year INTEGER, date VARCHAR) time VARCHAR) question= What is the time number of the ) when the is d"?"? and when the is "d"?"? and"? and"? Assistant:select count(ROUND) from TABLE_NAME_54 where PLAYER_RE "S"" and PLAYER = "J""" "" "<|end_of_text|>
0 2199 7500 1.5105022192001343 9.997779521644109e-05
Human: context= CREATE TABLE table_name_86 (date_ VARCHAR, date VARCHAR) question= What is the result the in is in a game 2-1-69- What? Assistant:select SCOREURN from TABLE_NAME_86 where SCORE = "198 - 200 - 200 - "8<|end_of_text|>
0 2299 7500 1.0620373487472534 1.002220478354209e-05
Human: context=

0 4299 7500 2.2276456356048584 1.0022204783542074e-05
Human: context= CREATE TABLE table (name_id VARCHAR, emp_id VARCHAR) CREATE TABLE staff VARCHAR (name_id VARCHAR, question= What many courses are the have the "3?? Assistant:select count(*) from pNAME_NAMEYS asCTIONS as t1 join C tER as as t2 on t1.PRORTER) = t2.SRTER) where t2.SRTER) = "1"<|end_of_text|>
0 4399 7500 1.067452311515808 9.997779521642105e-05
Human: context= CREATE TABLE table_name197772_1 (high VARCHAR, date VARCHAR VARCHAR VARCHAR) question= What the total number for  Assistant  Assistant Assistant Assistant Assistant Assistant:SELECT MIN() FROM table_112772_2 WHERE  = = = "8"2"142""""<|end_of_text|>
0 4499 7500 1.058398962020874 1.0022204783542054e-05
Human: context= CREATE TABLE table_name1348309_1 (date_e VARCHAR2 VARCHARm__ VARCHAR, population__) question= What  the episode elected ( in the 3. Assistant:SELECT us_e__2__m___ FROM table_2578309_2 WHERE series = 9<|end_of_text|>
0 4599 7500 1.4659454822540283 9.9977

0 6799 7500 1.3091473579406738 9.997779521642082e-05
Human: context= CREATE TABLE table (Name_id VARCHAR, course VARCHAR) course_id VARCHAR); question= What the course age of the who all course. are the course.? Assistant:select avgPARTMENT_ID, p(ERITY) from pLOYEES where pMISSION_IDIL = "n"<|end_of_text|>
0 6899 7500 0.7621736526489258 1.002220478354206e-05
Human: context= CREATE TABLE table_name1144401_1 (date_code VARCHAR, us VARCHARnumber VARCHAR) question= What is the highest code for the 1? the series? Assistant:select MIN(__) FROM table_15716419_3 WHERE series__number = 31<|end_of_text|>
0 6999 7500 0.6736539006233215 9.997779521640043e-05
Human: context= CREATE TABLE table_name_11 (total_team VARCHAR, venue_team VARCHAR) question= What away team has the D???? Assistant:SELECT away_team FROM table_name_95 WHERE home_team = "coll"ilton"<|end_of_text|>
0 7099 7500 0.4945829510688782 1.0022204783542085e-05
Human: context= CREATE TABLE table_name_62 (score VARCHAR VARCHAR, year VARC

1 2199 7500 0.8202688694000244 1.0022204783542108e-05
Human: context= CREATE TABLE table_name_58 (date VARCHAR VARCHAR, goals VARCHARpercentage VARCHAR, goals VARCHAR) question= What many goals against are scored when there were  win of than 0,6, points were than 0? Assistant:select goals(goals_for) FROM table_name_62 WHERE points__percentage < 0.65<|end_of_text|>
1 2299 7500 0.44607970118522644 9.997779521666321e-05
Human: context= CREATE TABLE table_name13477138_1 (team VARCHAR, year VARCHAR) question= What  year of B" is 200. % what is the other? the? Assistant:SELECT COUNT FROM table_19077101_3 WHERE others = "7.7%"<|end_of_text|>
1 2399 7500 0.7839294672012329 1.0022204783542174e-05
Human: context= CREATE TABLE table_name13422212_1 (date_by VARCHARap VARCHAR, year__) question= Who was of the episode of-lapril the 2.7 million?? Assistant:SELECT WR_BY_PLAYAPACITY TABLE_11622212_2 where VIEWERS = "13.4"ION"<|end_of_text|>
1 2499 7500 0.9817774891853333 9.997779521646167e-05
Human: co

...

3 3299 7500 0.6563590168952942 9.891625428786739e-05
Human: context= CREATE TABLE table_name1340834_2 (pick_air_date VARCHAR, written_no VARCHAR) question= What all episode3st air date for production production with a productionc productione production code. Assistant:SELECT ORIGINAL_AIRDATE from TABLE_2501754_3 where PROD_CODE = "1cRYe"<|end_of_text|>
3 3399 7500 0.8444675803184509 1.1083745712764291e-05
Human: context= CREATE TABLE table_name_87 (year INTEGER2 VARCHAR, week_2 VARCHAR) question= What is the week 2 with to week cyber the Alyse was a 3? Assistant:SELECT week_2 FROM table_name_60 WHERE week_3 = "thetied alyse"<|end_of_text|>
3 3499 7500 0.6519173979759216 9.891625428734654e-05
Human: context= CREATE TABLE table_name_37 (date VARCHAR, date VARCHAR) date VARCHAR) question= What is the record team's record on the played the New Rays on February 6? Assistant:SELECT record FROM table_name_50 WHERE opponent = "mont rays" AND date = "september 6"<|end_of_text|>
3 3599 7500 0.60

3 5899 7500 0.6624239087104797 9.858624225106924e-05
Human: context= CREATE TABLE table_name_68 (yearles VARCHAR, winss VARCHAR, poles VARCHAR) team_laps VARCHAR) poles VARCHAR) question= What Poles of Wins Winsest Laps of 0, and a Races of 0, and a Pod of than 0, and a Podiums smaller than 0? Assistant:SELECT MIN(poles) FROM table_name_74 WHERE races_laps = 0 AND races = 0 AND wins > 0 AND podiums < 2<|end_of_text|>
3 5999 7500 0.5357757210731506 1.1413757749216283e-05
Human: context= CREATE TABLE table_name13414361_1 (j VARCHAR, stadium__) question= What the is 5.9%, what is the city? Assistant:SELECT city FROM table_25214361_9 WHERE democratic = "42.5%"<|end_of_text|>
3 6099 7500 0.525014340877533 9.858624225055296e-05
Human: context= CREATE TABLE table_name_79 (record VARCHAR, rank VARCHAR, question= What is ranked rank9 as rank rank? Assistant:SELECT ELEVED from TABLE_NAME_21 where RANK = 5<|end_of_text|>
3 6199 7500 0.6412259340286255 1.1413757749208268e-05
Human: context= CREATE

4 999 7500 0.47991859912872314 9.821321585604135e-05
Human: context= CREATE TABLE table_name_28 (date_remaining INTEGER, round VARCHAR, question= What is the lowest of clubs remaining in the round round? Assistant:select max(UBS_REMAINING) from TABLE_NAME_65 where ROUND = "SECOND ROUND"<|end_of_text|>
4 1099 7500 0.31144917011260986 1.1786784144550709e-05
Human: context= CREATE TABLE table_name_35 (date_bonus VARCHAR, question= What a  of  what was the losing bonus? Assistant:select LOSING_BONUS from TABLE_NAME_52 where "LOST" = "LOST"<|end_of_text|>
4 1199 7500 0.41621503233909607 9.821321585552316e-05
Human: context= CREATE TABLE table_name_35 (op VARCHAR, pick VARCHAR) question= What player has up  matches10 w:SELECT player FROM table_name_78 WHERE matches = 9"<|end_of_text|>
4 1299 7500 0.3010824918746948 1.1786784144539088e-05
Human: context= CREATE TABLE table_name_69 (year VARCHAR, time VARCHAR) question= What the  that lasted 3:11, what was the result? Assistant:SELECT RES from

4 3699 7500 0.4111693501472473 1.2202456766741297e-05
Human: context= CREATE TABLE table_name_45 (date INTEGER, events VARCHAR, points VARCHAR) question= What many wins did thees have in 1986? Assistant:select COUNT(wins) FROM table_name_43 WHERE team = "196 c" AND year < 1962<|end_of_text|>
4 3799 7500 0.5610462427139282 9.779754323300869e-05
Human: context= CREATE TABLE table_name13485841_5 (high_re VARCHAR, location VARCHAR) question= Who had the most points in game 1? Assistant:select high_points FROM table_23211041_5 WHERE game = 4<|end_of_text|>
4 3899 7500 0.42464590072631836 1.2202456766712875e-05
Human: context= CREATE TABLE table_name13445530_1 (op VARCHAR, election VARCHAR) question= Who candidate every candidate of the candidate from Northfield, Illinois?? Assistant:SELECT CANDIDATE from TABLE_24501530_1 where HOMETOWN = "mARNE c, iESTERIALROALIA"<|end_of_text|>
4 3999 7500 0.46376439929008484 9.779754323312603e-05
Human: context= CREATE TABLE table_name_34 (score VARCHAR, 

4 6399 7500 0.41933473944664 9.733963460296258e-05
Human: context= CREATE TABLE table_name_52 (score_for INTEGER, played__percentage VARCHAR) points VARCHAR) question= How many goals have were scored under the were a percentage of than 0.0 and points of than 0? Assistant:SELECT COUNT(goals_for) FROM table_name_62 WHERE pct__percentage < 0.519 AND points > 65<|end_of_text|>
4 6499 7500 0.5516005158424377 1.2660365397062077e-05
Human: context= CREATE TABLE table_name_67 (date VARCHAR, date VARCHAR) question= What Reference for a RN containing thiocyanato-10scn ) as a prefix? Assistant:select discovery FROM table_name_77 WHERE lifetime = "gamyanato- (scn)"<|end_of_text|>
4 6599 7500 0.4434831738471985 9.73396346030813e-05
Human: context= CREATE TABLE table_name134155_7 (status VARCHAR_season VARCHAR, original_air_date VARCHAR) question= What is number aired the series originally aired on March 19, 1988? Assistant:SELECT MIN(no_in_series) FROM table_228973_5 WHERE original_air_date = "Febr